In [2]:
from typing import List
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from ragbuilder.generation.utils import dummyRetriever

class dummyRetriever(BaseRetriever):
    """A toy retriever that contains the top k documents that contain the user query.

    This retriever only implements the sync method _get_relevant_documents.

    If the retriever were to involve file access or network access, it could benefit
    from a native async implementation of `_aget_relevant_documents`.

    As usual, with Runnables, there's a default async implementation that's provided
    that delegates to the sync implementation running on another thread.
    """

    documents: List[Document]
    """List of documents to retrieve from."""
    k: int
    """Number of top results to return"""

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Sync implementations for retriever."""
        matching_documents = []
        for document in self.documents:
            if len(matching_documents) > self.k:
                return matching_documents

            if query.lower() in document.page_content.lower():
                matching_documents.append(document)
        return matching_documents

In [3]:
strings = [
    "A young woman.",
    """## Chunk 3: The Locket’s Secret

Clara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.

“Who are you?” she stammered, clutching her locket.

“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”
"""
]
documents = dummyRetriever.strings_to_documents(strings)
retriever = dummyRetriever(documents=documents, k=3)

AttributeError: type object 'dummyRetriever' has no attribute 'strings_to_documents'

In [17]:
retriever.invoke("Clara")

[Document(page_content='## Chunk 3: The Locket’s Secret\n\nClara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.\n\n“Who are you?” she stammered, clutching her locket.\n\n“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”\n')]